# Lab 12-2 many to one stacking
### sentence sentiment classification 
* many to one
* variable input sequence length
* stacking
* drop out

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.nn import embedding_lookup
from tensorflow.nn.rnn_cell import *
from tensorflow.nn import dynamic_rnn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

1.12.0


### Prepairing dataset

In [2]:
sentences = ['What I cannot create, I do not understand.',
             'Intellecuals solve problems, geniuses prevent them',
             'A person who never made a mistake never tied anything new.',
             'The same equations have the same solutions.']
y_data = [1,0,0,1] # 1은 리처드파인만 한 말, 0은 아인슈타인이 한 말

In [3]:
char_set = ['<pad>'] + sorted(list(set(''.join(sentences))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(idx2char)
print(char2idx)

{0: '<pad>', 1: ' ', 2: ',', 3: '.', 4: 'A', 5: 'I', 6: 'T', 7: 'W', 8: 'a', 9: 'b', 10: 'c', 11: 'd', 12: 'e', 13: 'g', 14: 'h', 15: 'i', 16: 'k', 17: 'l', 18: 'm', 19: 'n', 20: 'o', 21: 'p', 22: 'q', 23: 'r', 24: 's', 25: 't', 26: 'u', 27: 'v', 28: 'w', 29: 'y'}
{'<pad>': 0, ' ': 1, ',': 2, '.': 3, 'A': 4, 'I': 5, 'T': 6, 'W': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'g': 13, 'h': 14, 'i': 15, 'k': 16, 'l': 17, 'm': 18, 'n': 19, 'o': 20, 'p': 21, 'q': 22, 'r': 23, 's': 24, 't': 25, 'u': 26, 'v': 27, 'w': 28, 'y': 29}


In [4]:
x_data = list(map(lambda sentence : [char2idx.get(char) for char in sentence], sentences))
x_data_len = list(map(lambda sentence : len(sentence), sentences))

print(x_data)
print(x_data_len)

[[7, 14, 8, 25, 1, 5, 1, 10, 8, 19, 19, 20, 25, 1, 10, 23, 12, 8, 25, 12, 2, 1, 5, 1, 11, 20, 1, 19, 20, 25, 1, 26, 19, 11, 12, 23, 24, 25, 8, 19, 11, 3], [5, 19, 25, 12, 17, 17, 12, 10, 26, 8, 17, 24, 1, 24, 20, 17, 27, 12, 1, 21, 23, 20, 9, 17, 12, 18, 24, 2, 1, 13, 12, 19, 15, 26, 24, 12, 24, 1, 21, 23, 12, 27, 12, 19, 25, 1, 25, 14, 12, 18], [4, 1, 21, 12, 23, 24, 20, 19, 1, 28, 14, 20, 1, 19, 12, 27, 12, 23, 1, 18, 8, 11, 12, 1, 8, 1, 18, 15, 24, 25, 8, 16, 12, 1, 19, 12, 27, 12, 23, 1, 25, 15, 12, 11, 1, 8, 19, 29, 25, 14, 15, 19, 13, 1, 19, 12, 28, 3], [6, 14, 12, 1, 24, 8, 18, 12, 1, 12, 22, 26, 8, 25, 15, 20, 19, 24, 1, 14, 8, 27, 12, 1, 25, 14, 12, 1, 24, 8, 18, 12, 1, 24, 20, 17, 26, 25, 15, 20, 19, 24, 3]]
[42, 50, 58, 43]


In [5]:
# padding
max_sequence = 55
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence,
                       padding = 'post', truncating = 'post')

# 데이터 형태 확인
print(x_data)
print(x_data_len)
print(y_data)

[[ 7 14  8 25  1  5  1 10  8 19 19 20 25  1 10 23 12  8 25 12  2  1  5  1
  11 20  1 19 20 25  1 26 19 11 12 23 24 25  8 19 11  3  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 5 19 25 12 17 17 12 10 26  8 17 24  1 24 20 17 27 12  1 21 23 20  9 17
  12 18 24  2  1 13 12 19 15 26 24 12 24  1 21 23 12 27 12 19 25  1 25 14
  12 18  0  0  0  0  0]
 [ 4  1 21 12 23 24 20 19  1 28 14 20  1 19 12 27 12 23  1 18  8 11 12  1
   8  1 18 15 24 25  8 16 12  1 19 12 27 12 23  1 25 15 12 11  1  8 19 29
  25 14 15 19 13  1 19]
 [ 6 14 12  1 24  8 18 12  1 12 22 26  8 25 15 20 19 24  1 14  8 27 12  1
  25 14 12  1 24  8 18 12  1 24 20 17 26 25 15 20 19 24  3  0  0  0  0  0
   0  0  0  0  0  0  0]]
[42, 50, 58, 43]
[1, 0, 0, 1]


### Creating model

In [6]:
class Model:
    def __init__(self, x, x_len, y, hidden_dims, num_classes, dic):
        #input
        self.x = x
        self.x_len = x_len
        self.y = y
        self.dic = dic
        self.keep_prob = tf.placeholder(dtype=tf.float32)

        # token representation
        one_hot = tf.Variable(np.eye(len(self.dic)), dtype=tf.float32, trainable=False)
        x_batch = embedding_lookup(params=one_hot, ids=self.x)        

        # rnn
        rnn_cells = []
        for hidden_dim in hidden_dims:
            rnn_cells.append(DropoutWrapper(BasicRNNCell(num_units=hidden_dim),
                                            input_keep_prob=self.keep_prob))
        else:
            stacked_rnn_cell = MultiRNNCell(rnn_cells)
            
        _, state = dynamic_rnn(cell=stacked_rnn_cell, inputs=x_batch,
                               sequence_length=self.x_len, dtype=tf.float32)
        
        # output
        self.score = layers.Dense(units=num_classes)(state[-1])

        # loss
        self.ce_loss = tf.losses.sparse_softmax_cross_entropy(labels=self.y, logits=self.score)
            
        # prediction
        self.prediction = tf.argmax(input=self.score, axis=-1)
            
    def predict(self, sess, x, x_len, keep_prob = 1.):
        feed_prediction = {self.x : x, self.x_len : x_len, self.keep_prob : keep_prob}
        return sess.run(self.prediction, feed_dict=feed_prediction)

### Training model

In [7]:
# training hyper-paramter
lr = .01
epochs = 30
batch_size = 2

# generating data pipeline 
tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data, x_data_len))
tr_dataset = tr_dataset.shuffle(buffer_size=4)
tr_dataset = tr_dataset.batch(batch_size=2)

print(tr_dataset)
tr_iterator = tr_dataset.make_initializable_iterator()
x_mb, y_mb, x_mb_len = tr_iterator.get_next()

# creating model
model = Model(x=x_mb, y=y_mb, x_len=x_mb_len, hidden_dims=[10,10], num_classes=2, dic=char2idx)

<BatchDataset shapes: ((?, 55), (?,), (?,)), types: (tf.int32, tf.int32, tf.int32)>
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [8]:
# creating optimizer
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss=model.ce_loss)

In [9]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())

tr_loss_hist = []

for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0
    
    # training
    sess.run(tr_iterator.initializer)
    try:
        while True:
            _, tr_loss = sess.run([training_op, model.ce_loss], feed_dict = {model.keep_prob : .8})
            avg_tr_loss += tr_loss
            tr_step += 1
    except tf.errors.OutOfRangeError:
        avg_tr_loss /= tr_step
        
    if (epoch + 1) % 5 == 0:
        print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch+1, avg_tr_loss))

epoch :   5, tr_loss : 0.342
epoch :  10, tr_loss : 0.064
epoch :  15, tr_loss : 0.033
epoch :  20, tr_loss : 0.008
epoch :  25, tr_loss : 0.014
epoch :  30, tr_loss : 0.003


### Checking performance

In [10]:
yhat = model.predict(sess, x_data, x_data_len)

In [11]:
print('acc : {:.2%}'.format(np.mean(yhat == y_data)))

acc : 100.00%
